In [ ]:
!python --version

!pip install --upgrade pip

!pip install JPype1-1.1.2-cp38-cp38-win_amd64.whl

!pip install konlpy

In [ ]:

import pandas as pd 
import re
import matplotlib.pyplot  as plt
from konlpy.tag import Kkma
import konlpy
kkma = Kkma(max_heap_size= 1024 * 6)

## 데이터 통합

In [ ]:
call_list = [1,2,3,4,5,7,8,9,10,11,12,13,14,15]

In [ ]:
for i in call_list:
    globals()['globals_{}'.format(i)] = pd.read_csv('./tw크롤링 데이터/snsresultsampleAPI'+str(i)+'.csv', encoding = 'utf-8')


In [ ]:
#중복제거
for i in call_list:
    globals()['sample_{}'.format(i)] = globals()['globals_{}'.format(i)].drop_duplicates(['내용'])


In [ ]:
data = pd.concat([
    sample_1,sample_2,sample_3,sample_4, sample_5,
        sample_7,sample_8,sample_9,sample_10,sample_11,sample_12,
                  sample_13,sample_14,sample_15
])
data.fillna(value = "FILLVALUE")

data['토큰화']=0
data['추출단어']= ""
data['감성지수']=0
data.info()

 
 ### from konlpy.tag import Kkma
 kkma = Kkma()
 
 kkma.morphs         #형태소 분석
 
 kkma.nouns          #명사 분석
 
 kkma.pos            #형태소 분석 태깅
 
 kkma.sentences      #문장 분석
 
 
 ### 사용예시
 kkma.morphs(text)
 
 '공부', '를', '하', '면', '하', 'ㄹ수록', '모르', '는', '것', '이', '많', 다는', '것', '을', '알', '게', '되', 'ㅂ니다', '.'
 
 kkma.nouns(text)
 
 '대학', '통계학', '이산', '이산수학', '수학', '등'
 
 kkma.pos(text)
 
 ('다', 'MAG'), ('까먹', 'VV'), ('어', 'ECD'), ('버리', 'VXV'), ('었', 'EPT'), ('네요', 'EFN'), ('?', 'SF'), ('ㅋㅋ', 'EMO')
 
 kkma.sentences(text)
 
 '그래도 계속 공부합니다.', '재밌으니까!'

# Okt 
 - Okt(한국어 트위터 형태소 분석기)
 - kkma처리속도보다 현저히 빠름

In [ ]:
from konlpy.tag import Okt
okt = Okt(max_heap_size= 1024 * 6)


 ## okt.nouns(text)로 내용안에 명사를 추출

In [ ]:
errdictionary = []
import re
num = 0
errnum = 0

for i in range(len(data.iloc[:,1])):
    try:
        text = data.iloc[i,1]
        
        if type(text) != float:
            wordlist = okt.nouns(text)
            num+=1
            strwordlist = ""
            
            for word in wordlist:       
                if len(word) != 1:
                    strwordlist = strwordlist+" "+word
            data.iloc[i,3] = strwordlist
            
        else:
            errdictionary.append(num)
            
        if num%10000 == 0:
            print(num/10000,"만번째 실행중") 
        
    except:
        print("data ",num,"번째에서 오류")
        

 #  단어(명사) 추출

In [ ]:

sentimentdictionary = []
errdictionary = []
dict_df = pd.DataFrame(columns=['인덱스','단어'])
import re
num = 0
errnum = 0

for text in data.iloc[:,1]:
    try:
        if type(text) != float:
            wordlist = okt.nouns(text)
            num+=1
        else:
            errdictionary.append(num)
        if num%10000 == 0:
            print(num/10000,"만번째 실행중") 
        
        for word in wordlist:

            if len(word) != 1:
                sentimentdictionary.append(word)
    except:
        print("data ",num,"번째에서 오류")  
        
   
data_df = pd.DataFrame(sentimentdictionary, columns = ['단어'])

# 모든 특수문자와 숫자 제거
data_df["단어"] = data_df["단어"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
data_df["단어"] = data_df["단어"].str.replace('\d+', '')
            
#중복제거
dict_df = pd.DataFrame(data_df['단어'].unique(), columns = ['단어'])

dict_df['빈도수']=0
#40만개의 트윗에서 명사로 추정되는 단어 200만개를 추출
#200만개의 단어는 총 8만개의 단어로 이루어져있음
#dict_df['빈도수']에 단어별 쓰인 횟수를 저장

 ## 단어 빈도수 체크


In [ ]:
num = 0
for i in range(len(dict_df['단어'])):
    dict_df.iloc[i,1] = len(data_df[data_df['단어'] == dict_df.iloc[i,0]])
    num+=1
    if num%10000 == 0:
        print(num/10000,"만번째 실행중")

In [ ]:
#사전단어를 csv로 저장

dict_df.to_csv('dict_of_count.csv', index=False, encoding='utf-8-sig')

In [ ]:
#빈도수로 정렬 많이 쓰인 단어부터 감성수치 라벨링으로 신뢰도 업데이트후 불러오기
scoredict = pd.read_csv('./dict_of_count(sort).csv')

scoredict = scoredict.fillna(value = 0)
scoredict.info()

## 트윗별 감성스코어 확인

In [ ]:
num= 0
for i in range(len(data.iloc[:,3])):
    try:
        #okt.nouns로 추출한 단어 스플릿
        vv_word_list = data.iloc[i,3].split()
        score = 0
        num+=1
        if num%10000 == 0:
            print(num/10000,"만번째 실행중")
            
        if len(vv_word_list) !=0:
            for j in range(len(vv_word_list)):
                vv_word = vv_word_list[j]
                score_sum_df = scoredict[scoredict['단어'] == vv_word] ## df[조건식]
                temp= score_sum_df['라벨링']
                #단어마다 감성스코어를 확인후 점수합산
                if len(temp) !=0:
                    score += int(temp)
    except:
        
        #오류나는 원인 분석하니 트윗중 불용어를 제외하면
        #내용이없는 경우(7~8%정도) data['추출단어']가 널값이라 오류났음
        #이런것들은 0점처리
        print(num,"번째에서 오류")
        num+=1
        score = 0
    data.iloc[i,4]= score

In [ ]:
data.head()

In [ ]:
data.to_csv('data_emo_score.csv', index=False, encoding='utf-8-sig')

## 최종 감성지수(기업별)

In [ ]:
num = 0
list = data['종목'].unique()
result_emo_score = pd.DataFrame(columns=['기업명','감성지수'])
for i in range(len(list)):
    cop = list[i]
    temp_df = data[data['종목']==cop]
    score_sum = temp_df['감성지수'].sum()
    data_to_insert = {'기업명': cop, '감성지수': score_sum}
    result_emo_score = result_emo_score.append(data_to_insert, ignore_index=True)
    num += 1
    if num%100 == 0:
        print(num/100,"백번째 실행중")
result_emo_score_sort = result_emo_score.sort_values(by=['감성지수'], axis=0, ascending=False)

In [ ]:
result_emo_score_sort.to_csv('result_emo_score_sort.csv', index=False, encoding='utf-8-sig')